In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


y_features = pd.read_csv("../Data/cars_agg_label.csv")
y_features

,Unnamed: 0,fileNumber,xmin,ymin,xmax,ymax,class_no,class_label,train_test,subclass,Label1,Label1_No,Label2,Label2_No
0,0,1,112,7,853,717,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3
1,1,2,48,24,441,202,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3
2,2,3,7,4,277,180,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3
3,3,4,33,50,197,150,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3
4,4,5,5,8,83,58,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16180,16180,16181,38,36,375,234,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1
16181,16181,16182,29,34,235,164,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1
16182,16182,16183,25,32,587,359,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1
16183,16183,16184,56,60,208,186,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1


In [2]:
xtrain, xtest, ytrain, ytest = train_test_split(y_features.loc[:,"fileNumber"],
                                                y_features.loc[:,["fileNumber", "class_no", "Label1_No", "Label2_No"]],
                                                test_size=0.2,
                                                random_state=333)
ytrain

,fileNumber,class_no,Label1_No,Label2_No
12430,12431,151,1,1
15062,15063,183,2,2
7406,7407,91,3,4
7697,7698,94,3,3
36,37,1,3,3
...,...,...,...,...
6067,6068,75,3,4
9635,9636,118,3,3
973,974,13,1,1
14092,14093,171,1,1


In [3]:
def groupClasses(cars_df):
    class1 = "class_no"
    class2 = "Label1_No"
    class3 = "Label2_No"
    
    cars_df["count"] = 1
    new_df = cars_df.groupby([class1, class2, class3]).agg({"count": "count"})
    sumTotal = new_df["count"].sum()
    new_df["dist"] = new_df["count"]/sumTotal
    
    return new_df.reset_index()

trainGroup = groupClasses(ytrain)
testGroup = groupClasses(ytest)

In [4]:
print(trainGroup.groupby(["Label1_No"]).agg({"count":"sum", "dist":"sum"}))
print(trainGroup.groupby(["Label2_No"]).agg({"count":"sum", "dist":"sum"}))

           count      dist
Label1_No                 
1           4163  0.321517
2           4347  0.335728
3           4438  0.342756
           count      dist
Label2_No                 
1           4163  0.321517
2           4347  0.335728
3           2389  0.184507
4           1183  0.091365
5            866  0.066883


In [5]:
print(testGroup.groupby(["Label1_No"]).agg({"count":"sum", "dist":"sum"}))
print(testGroup.groupby(["Label2_No"]).agg({"count":"sum", "dist":"sum"}))

           count      dist
Label1_No                 
1           1031  0.318505
2           1046  0.323139
3           1160  0.358357
           count      dist
Label2_No                 
1           1031  0.318505
2           1046  0.323139
3            619  0.191226
4            331  0.102255
5            210  0.064875


In [6]:
print(trainGroup.groupby(["class_no"]).agg({"count":"sum", "dist":"sum"}))

          count      dist
class_no                 
1            67  0.005175
2            49  0.003784
3            72  0.005561
4            72  0.005561
5            66  0.005097
...         ...       ...
192          68  0.005252
193          72  0.005561
194          76  0.005870
195          69  0.005329
196          61  0.004711

[196 rows x 2 columns]


In [7]:
print(testGroup.groupby(["class_no"]).agg({"count":"sum", "dist":"sum"}))

          count      dist
class_no                 
1            22  0.006796
2            15  0.004634
3            14  0.004325
4            12  0.003707
5            15  0.004634
...         ...       ...
192          17  0.005252
193          11  0.003398
194          15  0.004634
195          17  0.005252
196          19  0.005870

[196 rows x 2 columns]


In [8]:
ytest

,fileNumber,class_no,Label1_No,Label2_No,count
8798,8799,108,3,5,1
5067,5068,63,2,2,1
13207,13208,161,1,1,1
7648,7649,94,3,3,1
11276,11277,137,2,2,1
...,...,...,...,...,...
3894,3895,49,2,2,1
10830,10831,131,3,3,1
662,663,9,1,1,1
4398,4399,55,1,1,1


In [9]:
y_features = pd.merge(y_features, ytest[["fileNumber", "count"]], how = "left", on = "fileNumber")
y_features.rename(columns={"count":"trn0tst1"}, inplace=True)
y_features.fillna(0, inplace = True)

In [10]:
y_features = y_features.astype({"trn0tst1": "int32"}, copy=False)

In [11]:
y_features

,Unnamed: 0,fileNumber,xmin,ymin,xmax,ymax,class_no,class_label,train_test,subclass,Label1,Label1_No,Label2,Label2_No,trn0tst1
0,0,1,112,7,853,717,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3,0
1,1,2,48,24,441,202,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3,0
2,2,3,7,4,277,180,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3,0
3,3,4,33,50,197,150,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3,1
4,4,5,5,8,83,58,1,AM General Hummer SUV 2000,0,u,SUVs/Trucks/Vans,3,SUV,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16180,16180,16181,38,36,375,234,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1,0
16181,16181,16182,29,34,235,164,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1,0
16182,16182,16183,25,32,587,359,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1,0
16183,16183,16184,56,60,208,186,196,smart fortwo Convertible 2012,1,c,Coupe/Convrtbl,1,Coupe/Convrtbl,1,0


In [12]:
y_features["trn0tst1"].sum()

3237

In [13]:
y_features.to_csv("../Data/cars_labels_train_test.csv")